In [ ]:
import keras
import numpy as np
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, AveragePooling2D
from keras.initializers import RandomNormal  
from keras import optimizers
from keras.callbacks import LearningRateScheduler, TensorBoard
from keras.layers.normalization import BatchNormalization

batch_size    = 128
epochs        = 200
iterations    = 391
num_classes   = 10
dropout       = 0.5
weight_decay  = 0.0001
log_filepath  = './nin'

def color_preprocessing(x_train,x_test):
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    mean = [125.307, 122.95, 113.865]
    std  = [62.9932, 62.0887, 66.7048]
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

    return x_train, x_test

def scheduler(epoch):
    if epoch <= 80:
        return 0.01
    if epoch <= 140:
        return 0.005
    return 0.001

def build_model():
  model = Sequential()

  model.add(Conv2D(192, (5, 5), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal", input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(160, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(96, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
  
  model.add(Dropout(dropout))
  
  model.add(Conv2D(192, (5, 5), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(192, (1, 1),padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(192, (1, 1),padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(3, 3),strides=(2,2),padding = 'same'))
  
  model.add(Dropout(dropout))
  
  model.add(Conv2D(192, (3, 3), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(192, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  model.add(Conv2D(10, (1, 1), padding='same', kernel_regularizer=keras.regularizers.l2(weight_decay), kernel_initializer="he_normal"))
  model.add(Activation('relu'))
  
  model.add(GlobalAveragePooling2D())
  model.add(Activation('softmax'))
  
  sgd = optimizers.SGD(lr=.1, momentum=0.9, nesterov=True)
  model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
  return model

if __name__ == '__main__':

    # load data
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    x_train, x_test = color_preprocessing(x_train, x_test)

    # build network
    model = build_model()
    print(model.summary())

    # set callback
    tb_cb = TensorBoard(log_dir=log_filepath, histogram_freq=0)
    change_lr = LearningRateScheduler(scheduler)
    cbks = [change_lr,tb_cb]

    # set data augmentation
    print('utilisant le real-time data augmentation !!')
    datagen = ImageDataGenerator(horizontal_flip=True,width_shift_range=0.125,height_shift_range=0.125,fill_mode='constant',cval=0.)
    datagen.fit(x_train)

    # start training
    model.fit_generator(datagen.flow(x_train, y_train,batch_size=batch_size),steps_per_epoch=iterations,epochs=epochs,callbacks=cbks,validation_data=(x_test, y_test))
    model.save('nin.h5')


Using TensorFlow backend.


170057728/170498071 [============================>.] - ETA: 0s_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 192)       14592     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 192)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 160)       30880     
_________________________________________________________________
activation_2 (Activation)    (None, 32, 32, 160)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 96)        15456     
_________________________________________________________________
activation_3 (Activation)    (None, 32, 32, 96)        0         
_________________________________________________________________
max_pooling2d_

391/391 [==============================] - 59s - loss: 0.6609 - acc: 0.8410 - val_loss: 0.6796 - val_acc: 0.8360
Epoch 37/200
391/391 [==============================] - 59s - loss: 0.6492 - acc: 0.8437 - val_loss: 0.6672 - val_acc: 0.8405
Epoch 38/200
391/391 [==============================] - 59s - loss: 0.6423 - acc: 0.8470 - val_loss: 0.6760 - val_acc: 0.8374
Epoch 39/200
391/391 [==============================] - 59s - loss: 0.6368 - acc: 0.8455 - val_loss: 0.6599 - val_acc: 0.8460
Epoch 40/200
391/391 [==============================] - 59s - loss: 0.6336 - acc: 0.8486 - val_loss: 0.6579 - val_acc: 0.8428
Epoch 41/200
391/391 [==============================] - 59s - loss: 0.6233 - acc: 0.8519 - val_loss: 0.6698 - val_acc: 0.8430
Epoch 42/200
391/391 [==============================] - 59s - loss: 0.6161 - acc: 0.8550 - val_loss: 0.7027 - val_acc: 0.8300
Epoch 43/200
391/391 [==============================] - 59s - loss: 0.6127 - acc: 0.8550 - val_loss: 0.6441 - val_acc: 0.8500
Epoch

391/391 [==============================] - 59s - loss: 0.4076 - acc: 0.9224 - val_loss: 0.5737 - val_acc: 0.8755
Epoch 102/200
391/391 [==============================] - 59s - loss: 0.4060 - acc: 0.9220 - val_loss: 0.5628 - val_acc: 0.8819
Epoch 103/200
391/391 [==============================] - 59s - loss: 0.4011 - acc: 0.9241 - val_loss: 0.5574 - val_acc: 0.8831
Epoch 104/200
391/391 [==============================] - 59s - loss: 0.4039 - acc: 0.9227 - val_loss: 0.5627 - val_acc: 0.8830
Epoch 105/200
391/391 [==============================] - 59s - loss: 0.4009 - acc: 0.9229 - val_loss: 0.5715 - val_acc: 0.8766
Epoch 106/200
391/391 [==============================] - 59s - loss: 0.4021 - acc: 0.9248 - val_loss: 0.5562 - val_acc: 0.8807
Epoch 107/200
391/391 [==============================] - 59s - loss: 0.3986 - acc: 0.9241 - val_loss: 0.5667 - val_acc: 0.8798
Epoch 108/200
391/391 [==============================] - 59s - loss: 0.3967 - acc: 0.9249 - val_loss: 0.5824 - val_acc: 0.877

391/391 [==============================] - 58s - loss: 0.3166 - acc: 0.9513 - val_loss: 0.5543 - val_acc: 0.8942
Epoch 166/200
391/391 [==============================] - 58s - loss: 0.3187 - acc: 0.9509 - val_loss: 0.5416 - val_acc: 0.8921
Epoch 167/200
391/391 [==============================] - 58s - loss: 0.3143 - acc: 0.9516 - val_loss: 0.5477 - val_acc: 0.8927
Epoch 168/200
391/391 [==============================] - 58s - loss: 0.3180 - acc: 0.9505 - val_loss: 0.5554 - val_acc: 0.8920
Epoch 169/200
391/391 [==============================] - 58s - loss: 0.3161 - acc: 0.9515 - val_loss: 0.5567 - val_acc: 0.8918
Epoch 170/200
391/391 [==============================] - 58s - loss: 0.3173 - acc: 0.9508 - val_loss: 0.5496 - val_acc: 0.8906
Epoch 171/200
391/391 [==============================] - 58s - loss: 0.3148 - acc: 0.9525 - val_loss: 0.5418 - val_acc: 0.8918
Epoch 172/200
391/391 [==============================] - 58s - loss: 0.3125 - acc: 0.9526 - val_loss: 0.5700 - val_acc: 0.887